# An introduction to ExemPy (formerly GCMPy)

In [1]:
%load_ext autoreload
from ExemPy import *
from ExemPy.utils import *
from ExemPy.viz import *
from ExemPy.GCM import *
%aimport ExemPy, ExemPy.utils, ExemPy.viz, ExemPy.GCM
%autoreload 1
import math
import random
import matplotlib.pyplot as plt
#%matplotlib inline
import numpy as np
import pandas as pd
from pandas import DataFrame
from scipy.optimize import minimize
import seaborn as sns
sns.set(style='ticks', context='paper')
colors=["#e3c934","#68c4bf","#c51000","#287271"]
sns.set_palette(colors)

In [2]:
pb52 = pd.read_csv('pb52_data//pb52.csv')
pbcm = pd.read_csv('pb52_data//pbcm.csv').drop([0]).set_index('vowelChoice').rename_axis(None)
pb52 = HzToBark(pb52, ["F0", "F1", "F2", "F3"])
pb52.sample(5)

,type,gender,speaker,vowel,repetition,F0,F1,F2,F3,z0,z1,z2,z3
930,w,f,47,PALM,1,252,900,1290,2750,2.524304,7.906713,10.111508,15.123397
668,w,f,34,STRUT,1,227,635,1200,3250,2.252748,6.030443,9.651013,16.194088
1299,c,f,65,NURSE,2,265,610,2100,2600,2.663101,5.833463,13.337241,14.756404
55,m,m,3,FOOT,2,114,456,1030,2300,0.943645,4.530166,8.705552,13.944883
1330,c,f,67,PALM,1,218,1090,1380,3050,2.153462,9.051279,10.547186,15.791457


In [3]:
catslist = ["type", "vowel"]
cval = 25
exemplars = pb52
test = pb52

In [4]:
dims1 = {'z0' : 1,
         'z1' : 3.585,
         'z2' : 2.246,
         'z3' : 2.736}

dims2 = {'z0' : 1,
         'z1' : 2.72,
         'z2' : 1.322,
         'z3' : 0.882}

dims3 = {'z0' : 1,
         'z1' : 1.589,
         'z2' : 0.586,
         'z3' : 0.55}

dims4 = {'z0' : 1,
         'z1' : 2.534,
         'z2' : 1.891,
         'z3' : 1.784}

dims5 = {'z0' : 1,
         'z1' : 1.685,
         'z2' : 1.59,
         'z3' : 1.162}

In [5]:
weightslist = [dims1, dims2, dims3, dims4, dims5]

### Next, let's categorize!
This may take a minute!

In [6]:
pbflat = pd.Series(pbcm.stack(), name = "PB")

In [8]:
i = 1
rowlist=[]

for w in weightslist:
    choices = multicat(
        testset = test,
        cloud = exemplars,
        cats = catslist,
        dimsdict = w,
        c = cval,
        N = 1,
        exclude_self = True)
    
    cm = confusion(choices, catslist)['vowel']
    flat = pd.Series(cm.stack(), name = "GCM")
    matrices = pd.concat([pbflat, flat], axis=1)
    matrices.replace(0, np.nan)
    
    rms = (((matrices.PB-matrices.GCM) ** 2).mean()) ** .5
    r = r = matrices['PB'].corr(matrices['GCM'])
    
    print(i, rms, r)

    i += 1

1 0.07311066953598495 0.9644969811840054
2 0.07478482466383136 0.9636896723906305
3 0.07439469067077299 0.9662685200856319
4 0.07269092102869518 0.9654008646727258
5 0.07493043440418586 0.9658137424461452
